In [8]:
!pip uninstall -qqy jupyterlab  # Remove unused conflicting packages
!pip install -U -q "google-genai==1.7.0"
!pip install ipywidgets
!pip install pycountry geonamescache

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 56.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 57.2 MB/s eta 0:00:00:00:0100:01


In [9]:
from google import genai
from google.genai import types

genai.__version__

'1.7.0'

In [11]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
OPENWEATHER_API_KEY = UserSecretsClient().get_secret("OPENWEATHER_API_KEY")

In [13]:
# Define a retry policy. The model might make multiple consecutive calls automatically
# for a complex query, this ensures the client retries if it hits quota limits.
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

In [14]:
import requests

OPENWEATHER_URL = "https://api.openweathermap.org/data/2.5/weather"

def get_weather(location: str) -> dict:
    """Fetch current weather data for a given location using OpenWeatherMap API."""
    params = {
        "q": location,
        "appid": OPENWEATHER_API_KEY,
        "units": "metric"  # Change to 'imperial' for Fahrenheit
    }
    try:
        response = requests.get(OPENWEATHER_URL, params=params)
        response.raise_for_status()
        data = response.json()

        weather_info = {
            "location": data.get("name"),
            "temperature": data["main"]["temp"],
            "feels_like": data["main"]["feels_like"],
            "humidity": data["main"]["humidity"],
            "wind_speed": data["wind"]["speed"],
            "description": data["weather"][0]["description"].capitalize()
        }
        return weather_info

    except requests.RequestException as e:
        return {"error": str(e)}

In [18]:
from google import genai
from google.genai import types

client = genai.Client(api_key=GOOGLE_API_KEY)

instruction = """You are a helpful assistant that recommends outfits based on future weather forecasts.

Before recommending an outfit, use the `get_weather_forecast` tool to check the forecast for the location and time requested by the user.

Ask the user for both:
- The location (city/town)
- The number of hours into the future they want the forecast for

Then use that weather information to suggest what to wear."""

chat = client.chats.create(
    # model="gemini-1.5-flash",  # Or "gemini-2.0-flash"
    model="gemini-2.0-flash", 
    config=types.GenerateContentConfig(
        system_instruction=instruction,
        
        tools=[get_weather],  # Registers the tool
    )
)

In [20]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import geonamescache

# --- Load city data ---
gc = geonamescache.GeonamesCache()
cities_dict = gc.get_cities()
major_cities = sorted(set(city["name"] for city in cities_dict.values()))

# --- Unified input width and label style ---
label_width = "80px"
input_width = "300px"

# --- Widgets ---
city_dropdown = widgets.Combobox(
    options=major_cities,
    placeholder="Start typing a city...",
    description="City:",
    ensure_option=False,
    style={"description_width": label_width},
    layout=widgets.Layout(width=input_width)
)

custom_city_input = widgets.Text(
    placeholder="Or type another city",
    description="Custom:",
    style={"description_width": label_width},
    layout=widgets.Layout(width=input_width)
)

hours_slider = widgets.IntSlider(
    value=24,
    min=0,
    max=72,
    step=1,
    description='Hours:',
    style={"description_width": label_width},
    layout=widgets.Layout(width=input_width)
)

go_button = widgets.Button(
    description="Get Recommendation",
    button_style="success",
    layout=widgets.Layout(width=input_width)
)

output = widgets.Output()

# --- Button logic ---
def on_go_clicked(b):
    output.clear_output()
    location = custom_city_input.value.strip() or city_dropdown.value.strip()
    hours = hours_slider.value

    if not location:
        with output:
            print("⚠️ Please enter or select a city.")
        return

    with output:
        print(f"📍 Location: {location}")
        print(f"⏳ Forecast for {hours} hours later...\n")
        response = chat.send_message(f"What should I wear in {location} in {hours} hours?")
        print(response.text)

go_button.on_click(on_go_clicked)

# --- Display ---
form = widgets.VBox([
    city_dropdown,
    custom_city_input,
    hours_slider,
    go_button,
    output
])
display(form)

In [25]:
response = chat.send_message("What should I wear in Vancouver in 12 hours?")
print(response.text)

The weather in Vancouver in 12 hours is expected to be moderate rain with a temperature of 9.37 degrees Celsius, but it will feel like 6.56 degrees Celsius. The humidity will be 91% and the wind speed is 5.66 m/s.

I would recommend wearing a waterproof jacket, warm layers like a sweater or fleece, and waterproof shoes or boots. An umbrella would also be a good idea.

